# IE303

### Members’ Names: Asja Bašović, Hatidža Imamović, Sara Avdić, Hava Dedić
## Project Title: Optimization of room assignment for the exam schedule


# Important note
### All data was removed (censored) before upload as it is private property of the International University of Sarajevo
### All compromising outputs of cells of the said data was also cleared

In [ ]:
#imports needed for the model
import pulp
from datetime import datetime
from collections import defaultdict
import pandas as pd
import math

In [ ]:
# all of the data needed for the full model

#list of all acceptable rooms

## NOTE: Data was removed before upload  due to privacy reasons

room_to_building = {name: name.split()[0] for name in rooms}
buildings = sorted(set(room_to_building.values()))
#buildings needed to make sure big exams are scheduled ina  single building


# The optimization model

In [ ]:
# helper functions
def time_overlap(t1, t2):
    return not (t1[1] <= t2[0] or t2[1] <= t1[0])

def overlaps(i1, i2):
    return exam_day[i1] == exam_day[i2] and time_overlap(exam_times[i1], exam_times[i2])

def min_group_size(exam_size):
    return 0 if exam_size < 40 else 10

# model
def solve_exam_assignment( exams, rooms, c, s, buildings, room_to_building, exam_day, exam_times):
    prob = pulp.LpProblem("ExamRoomAssignment", pulp.LpMinimize)

    # decision variables
    # number of students from exam i assigned to room j
    x = pulp.LpVariable.dicts("x", [(i, j) for i in exams for j in rooms],lowBound=0, upBound=max(c.values()), cat='Integer')
    # binary variable indicating whether exam i uses room j
    w = pulp.LpVariable.dicts("w", [(i, j) for i in exams for j in rooms],cat='Binary')
    # binary variable indicating whether exam i is assigned to building b
    y = pulp.LpVariable.dicts("y", [(i, b) for i in exams for b in buildings],cat='Binary')

    # objective: minimize total unused seats
    prob += pulp.lpSum([c[j] - x[(i, j)] for i in exams for j in rooms])

    # Constraint 1: Assign all students
    for i in exams:
        prob += pulp.lpSum([x[(i, j)] for j in rooms]) == s[i]

    # Constraint 2: Avoid overbooking rooms for overlapping exams
    for j in rooms:
        for i1 in exams:
            overlapping = [i2 for i2 in exams if overlaps(i1, i2)]
            prob += pulp.lpSum([x[(i2, j)] for i2 in overlapping]) <= c[j]

   # Constraint 3a: Only one building per exam
    for i in exams:
        prob += pulp.lpSum([ y[(i, b)] for b in buildings ]) == 1

    # Constraint 3b: Link “x” to “y”: if exam i uses any of room j, room j must be in the chosen building
    for i in exams:
        for j in rooms:
            b = room_to_building[j]
            prob += x[(i, j)] <= c[j] * y[(i, b)]
            # if x[i,j] > 0 then y[i,b] must be 1 (so exam i is assigned to building b)

    # Constraint 4a and 4b: Min group size logic + Room Usage Linking Constraint
    for i in exams:
        for j in rooms:
            prob += x[(i, j)] <= c[j] * w[(i, j)]
            if s[i] >= 40:
                prob += x[(i, j)] >= 10 * w[(i, j)]
            else:
                prob += x[(i, j)] >= 1e-3 * w[(i, j)]

    # Constraint 5: Prevent splitting small exams
    for i in exams:
        if s[i] < 50:
             prob += pulp.lpSum([ w[(i, j)] for j in rooms ]) <= 1

    # solving
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    status = pulp.LpStatus[prob.status]
    objective_value = pulp.value(prob.objective)

    schedule_data = []
    for (i, j) in x:
        val = x[(i, j)].value()
        if val and val > 0:
            start, end = exam_times[i]
            #print(f"Assign {int(val)} students of exam {i} to room {j} on {exam_day[i]} from {start}:00 to {end}:00")
            schedule_data.append({
                'Exam': i,
                'Room': j,
                'Building': room_to_building[j],
                'Date': exam_day[i],
                'Start Time': f"{start}:00",
                'End Time': f"{end}:00",
                'Students Assigned': int(val),
                'Total Students for Exam': s[i],
                'Total exam capacity of the room': c[j]
            })


    schedule_df = pd.DataFrame(schedule_data)
    schedule_df = schedule_df.sort_values(by=['Date', 'Start Time', 'Room'])
    schedule_df.to_excel(r"C:\Users\Asja\Desktop\IE303_Files\exam_schedule_solution.xlsx", index=False)
    #print("Schedule saved to exam_schedule.xlsx")

    return status, objective_value,x, w, y

# Testing

### Checking what scaling factors still produce feasible solutions

In [ ]:
for a in [1.00, 1.01, 1.02, 1.05]:
    # scaling
    s_test = { i: math.ceil(a * s[i]) for i in exams }
    #print(s == s_test) #just to make sure the scaling is correct
    status_test, obj_test, x_vars, w_vars, y_vars = solve_exam_assignment(
        exams            = exams,
        rooms            = rooms,
        c                = c,
        s                = s_test,
        buildings        = buildings,
        room_to_building = room_to_building,
        exam_day         = exam_day,
        exam_times       = exam_times
    )

    print(f"α = {a:.2f} → Status = {status_test}", end="")
    if status_test == "Optimal":
        print(f" | Unused seats = {obj_test:.0f}")
    else:
        print("")

α = 1.00 → Status = Optimal | Unused seats = 227896
α = 1.01 → Status = Optimal | Unused seats = 227680
α = 1.02 → Status = Optimal | Unused seats = 227632
α = 1.05 → Status = Optimal | Unused seats = 227393


### Finding the maximum number of students the current model can make an optimal solution for

In [ ]:
def find_max_scaling(s_original, rooms, c, exams, buildings, room_to_building, exam_day, exam_times,
                     alpha_low=1.0, alpha_high=2.0, tol=0.01):   #uses binary search logic
    low = alpha_low
    high = alpha_high

    # check if high is already infeasible
    #if not, double until infeasible
    while True:
        s_high = {i: math.ceil(high * s_original[i]) for i in exams}
        status_high, _, _,_,_ = solve_exam_assignment(
            s = s_high,
            rooms = rooms,
            c = c,
            exams = exams,
            buildings = buildings,
            room_to_building = room_to_building,
            exam_day = exam_day,
            exam_times = exam_times
        )
        if status_high == "Infeasible":
            break
        high *= 2  # keep doubling until infeasible

    # binary search between low (feasible) and high (infeasible)
    while high - low > tol:
        mid = (low + high) / 2.0
        s_mid = {i: math.ceil(mid * s_original[i]) for i in exams}
        status_mid, _, _,_,_ = solve_exam_assignment(
            s = s_mid,
            rooms = rooms,
            c = c,
            exams = exams,
            buildings = buildings,
            room_to_building = room_to_building,
            exam_day = exam_day,
            exam_times = exam_times
        )

        if status_mid == "Optimal":
            low = mid
        else:
            high = mid

    return low


max_alpha = find_max_scaling(
    s_original = s,
    rooms = rooms,
    c = c,
    exams = exams,
    buildings = buildings,
    room_to_building = room_to_building,
    exam_day = exam_day,
    exam_times = exam_times,
    alpha_low = 1.0,
    alpha_high = 2.0,
    tol = 0.01
)
print(f"Maximum feasible scaling factor (approx): {max_alpha:.2f}x")

Maximum feasible scaling factor (approx): 2.01x


### Checking if removal of any single room leads to an infeasible solution

In [ ]:
columns = [
    "RemovedRoom",
    "Status",
    "TotalUnusedSeats",
    "Note"
]
results = []

for removed in rooms:
    rooms_reduced = [r for r in rooms if r != removed]

    c_reduced = { r: c[r] for r in rooms_reduced }

    status_r, obj_r, x_r, y_r, w_r = solve_exam_assignment(
        exams            = exams,
        rooms            = rooms_reduced,
        c          = c_reduced,
        s                = s,
        buildings        = buildings,
        room_to_building = room_to_building,
        exam_day         = exam_day,
        exam_times       = exam_times
    )

    if status_r == "Optimal":
        results.append({
            "RemovedRoom": removed,
            "Status": "Optimal",
            "TotalUnusedSeats": obj_r,
            "Note": ""
        })
    else:
        results.append({
            "RemovedRoom": removed,
            "Status": status_r,
            "TotalUnusedSeats": None,
            "Note": "Infeasible"
        })

df_results = pd.DataFrame(results, columns=columns)
df_results = df_results.sort_values(by=["Status", "TotalUnusedSeats"], ascending=[False, True]).reset_index(drop=True)
df_results.to_excel(r"C:\Users\Asja\Desktop\IE303_Files\roomTest.xlsx", index=False)
df_results

### Checking if only buliding A is sufficiant for an optimal model (the expected answer is no)

In [ ]:
rooms_test = [ # removed for privacy reasons
    

]

c_test = { # removed for privacy reasons

}
status_test, obj_test, x_vars, _, _ = solve_exam_assignment(
        exams = exams,
        rooms = rooms_test,
        c = c_test,
        s = s,
        buildings = buildings,
        room_to_building = room_to_building,
        exam_day = exam_day,
        exam_times = exam_times
    )
print('Model when only building A is used:', status_test)

Model when only building A is used: Infeasible


# Solution

In [ ]:
status1, obj1,x,w,y = solve_exam_assignment(exams, rooms, c, s,  buildings, room_to_building, exam_day, exam_times)
print("Full working model", status1, obj1)

Full working model Optimal 227896.0


In [ ]:
# calculating the actual unused seats
# track how many students are in each room at each time slot
assigned_per_room_slot = defaultdict(int)

for (i, j), var in x.items():
    val = var.value()
    if val and val > 0:
        start, end = exam_times[i]
        key = (j, exam_day[i], start, end)
        assigned_per_room_slot[key] += val

unused_per_room_slot = {}
total_unused = 0

print("Unused seats per used room per time slot:")
for (room, date, start, end), assigned in assigned_per_room_slot.items():
    capacity = c[room]
    unused = max(capacity - assigned, 0)
    unused_per_room_slot[(room, date, start, end)] = unused
    total_unused += unused
    print(f"Room {room} on {date} from {start}:00 to {end}:00 → {int(unused)} unused seats")

#print("Total unused seats in used rooms per time slot:", int(total_unused))

unused_data = []

for (room, date, start, end), unused in unused_per_room_slot.items():
    unused_data.append({
        'Room': room,
        'Date': date,
        'Start Time': f"{start}:00",
        'End Time': f"{end}:00",
        'Unused Seats': int(unused)
    })

unused_df = pd.DataFrame(unused_data)
unused_df = unused_df.sort_values(by=['Date', 'Start Time', 'Room'])
unused_df.to_excel(r"C:\Users\Asja\Desktop\IE303_Files\unused_seats4.xlsx", index=False)
print("Unused seat summary saved to unused_seats.xlsx")

In [ ]:
# checking the number of rooms used on each day
rooms_per_day = defaultdict(set)

for (i, j), var in x.items():
    val = var.value()
    if val and val > 0:
        day = exam_day[i]
        rooms_per_day[day].add(j)

print("Number of rooms assigned per day:")
for day, rooms in rooms_per_day.items():
    print(f"{day}: {len(rooms)} rooms")